In [32]:
# Model Name List
model_name_list = ["openai-community/gpt2-xl", "", "",
                   "meta-llama/Llama-2-7b", "meta-llama/Meta-Llama-3-8B", "meta-llama/Meta-Llama-3.1-8B-Instruct",
                   "internlm/internlm2_5-7b-chat", "microsoft/Phi-3-small-128k-instruct", "01-ai/Yi-1.5-9B-Chat"]

### Hugging Face

In [33]:
import transformers
import torch

model_name = model_name_list[5]
print("model_name: ", model_name)

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto', trust_remote_code = True)
"""
tokenizer.save_pretrained("./models/" + model_name)
model.save_pretrained("./models/" + model_name)

tokenizer = transformers.AutoTokenizer.from_pretrained("./models/" + model_name, trust_remote_code = True)
model = transformers.AutoModelForCausalLM.from_pretrained("./models/" + model_name, device_map = 'auto', trust_remote_code = True)
"""

pipeline = transformers.pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    model_kwargs={"torch_dtype": torch.float16}
)

model_name:  meta-llama/Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.34s/it]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [44]:
attention = "focus"
emotion = "surprise"

system_prompt = f"\
    You are a professional in UX usability evaluation designed to evaluate the user experience.\n\
    The user is currently experiencing it in real time, and you are conducting a UX evaluation in real time. \
    You should ask questions based on the \"Current User Status\" to evaluate various UX factors such as UI/UX design, user interaction, and user satisfaction.\n\
    \n\
    ## Answering Rules\n\
    1. Follow the \"Answering Form\"\n\
    2. Only generate questions for <UX EVALUATION>.\n\
    3. In <Report>, write a summary of the user's overall status and behavioral feedback to the user.\n\
    4. Do not regenerate the \"Current User Status\".\n\
    5. Generate the answer as a markdown language.\n\
    \n\
    ## User Status List\n\
    - Attention: [focus, unfocus, drowsy]\n\
    - Emotion: [angry, disgust, fear, happy, sad, surprise, neutral]\n\
    \n\
    ## Report Examples\n\
    Example 1 (When the user's \"Attention\" is in the drowsy state): You are dozing off now! Wake up!\n\
    \n\
    ## Question Examples\n\
    Example 1 (When the user's \"Attention\" is in the unfocus state): Did you feel like you had too many features or information?\n\
    Example 2 (When the user's \"Emotion\" is in the angry state): What part of your use did you feel stressed about?\n\
    Example 3 (When the user's \"Emotion\" is in the happy state): How satisfied were you after using it?\n\
    \n\
    ## Answering Form\n\
    <Report>\n\
    (Report)\n\
    \n\
    <UX EVALUATION>\n\
    Q: (question)\n\
    Q: (question)\n\
    Q: (question)\n\
    "

user_status = f"\
    ## Current User Status\n\
    - Attention: {attention}\n\
    - Emotion: {emotion}\n\
    "

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"{user_status}\nGenerate a user status report and the most effective UX evaluation questions for the current situation."},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"][-1]['content'])

## <Report>
The user is currently in a state of high focus and surprise. This suggests that they have just encountered something unexpected, and their attention is fully engaged. It's essential to understand the cause of their surprise and how it's affecting their interaction with the system.

## <UX EVALUATION>
Q: What just happened that caught your attention and caused you to feel surprised?
Q: Are you able to quickly understand the reason behind this unexpected event, or is it unclear to you?
Q: Is there anything that you find confusing or unclear about the current situation?


In [ ]:
import transformers
import torch

model_name = model_name_list[5]

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto', trust_remote_code = True)

"""
tokenizer.save_pretrained("./models/" + model_name)
model.save_pretrained("./models/" + model_name)

tokenizer = transformers.AutoTokenizer.from_pretrained("./models/" + model_name)
model = transformers.AutoModelForCausalLM.from_pretrained("./models/" + model_name)
"""

pipeline = transformers.pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    model_kwargs={"torch_dtype": torch.bfloat16}
)

### Open AI

In [43]:
from openai import OpenAI
from dotenv import load_dotenv

In [45]:
def generate_medical_report(system_prompt, user_status):
    load_dotenv()
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"{user_status}\nGenerate a user status report and the most effective UX evaluation questions for the current situation."},
        ],
        max_tokens=500,
        temperature=0,
        top_p=1.0,  # 토큰 확률의 상위 경계 설정
        n=1,  # 생성할 샘플의 수
        stop=None  # 생성 중단 조건
    )
    
    return response.choices[0].message.content

In [47]:
print(generate_medical_report(system_prompt, user_status))

<Report>
The user is currently focused and feeling surprised while using the interface.

<UX EVALUATION>
Q: What specific feature or aspect of the interface surprised you the most?
Q: Did the surprise enhance your overall user experience?
Q: How would you rate the clarity of the information provided on the interface that led to your surprise?


In [ ]:
for i in range(1, 11):
        contextFile = pd.read_csv(f'context/contexts_{i}_0429.csv')
        resultFile = pd.read_csv(f'faiss_result/results_{i}_0429.csv')
        reportFile = f'result/medical_report_{i}_0429.csv'
        results = []

        with open(reportFile, 'w') as f:

            for index, row in contextFile.iterrows():
                contextAll6 = row['contextAll6']

                print(i, "-", index)
                medicalReportAll = generate_medical_report(prompt, contextAll6)
                # print(medicalReportAll)

                results.append([contextFile.loc[index, 'query_study_id'],
                                contextFile.loc[index, 'query_subject_id'],
                                resultFile.loc[index, 'query_report'],
                                medicalReportAll])

        columns = ['query_study_id', 'query_subject_id', 'query_report', 'medicalReportAll']

        reportDF = pd.DataFrame(results, columns=columns)
        reportDF.to_csv(reportFile, index=False)